# 说明：

本例主要是**多输出**模型：

输出分为两大方向：1. 颜色类；2. 服饰类；

每个方向下又都存在多种标签：1. 颜色类有黑、蓝、红3种标签；2. 服饰类有牛仔裤、鞋子、连衣裙、衬衣4种标签。

---
数据集策略：先把所有数据一起读进来，并进行各种预处理操作。然后对所有数据集进行训练集、测试集的划分！

知识补充：
- 所有的层，都有一个name属性，可以用来给该层起名字！

### 获取所有文件路径、标签

In [1]:
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
import glob

In [2]:
# 所有文件的地址：
all_image_path = glob.glob('E:/tensorflow2.0_日月光华/日月光华-tensorflow资料/多输入多输出数据集/data/dataset/*/*.jpg')

In [3]:
len(all_image_path), type(all_image_path[0])

(2096, str)

In [4]:
all_image_path[0]

'E:/tensorflow2.0_日月光华/日月光华-tensorflow资料/多输入多输出数据集/data/dataset\\black_jeans\\00000000.jpg'

In [5]:
# 将数据乱序：
import random
random.shuffle(all_image_path)

# 查看乱序情况：
all_image_path[0:3]  

['E:/tensorflow2.0_日月光华/日月光华-tensorflow资料/多输入多输出数据集/data/dataset\\blue_dress\\00000333.jpg',
 'E:/tensorflow2.0_日月光华/日月光华-tensorflow资料/多输入多输出数据集/data/dataset\\blue_dress\\00000057.jpg',
 'E:/tensorflow2.0_日月光华/日月光华-tensorflow资料/多输入多输出数据集/data/dataset\\red_shirt\\00000202.jpg']

In [6]:
# 标签提取：还是按照\\来划分
all_image_labels = [ p.split('\\')[1] for p in all_image_path]
all_image_labels[0:3]

['blue_dress', 'blue_dress', 'red_shirt']

In [7]:
# 2级标签：颜色
all_color_labels = [ p.split('_')[0] for p in all_image_labels ]
# 2级标签：物品
all_item_labels = [ p.split('_')[1] for p in all_image_labels ]

In [8]:
# 标签-索引
pure_labels = glob.glob('E:/tensorflow2.0_日月光华/日月光华-tensorflow资料/多输入多输出数据集/data/dataset/*')
pure_labels = [ p.split('\\')[1] for p in pure_labels ]

pure_color_labels = set( [ p.split('_')[0] for p in pure_labels ] )  # set创建无序不重复元素集！
pure_item_labels = set( [ p.split('_')[1] for p in pure_labels ] )

pure_color_labels_to_index = dict( (name, index) for (index, name) in enumerate(pure_color_labels) )
pure_item_labels_to_index = dict( (name, index) for (index, name) in enumerate(pure_item_labels) )

In [9]:
pure_color_labels, pure_item_labels

({'black', 'blue', 'red'}, {'dress', 'jeans', 'shirt', 'shoes'})

In [10]:
pure_color_labels_to_index, pure_item_labels_to_index

({'red': 0, 'black': 1, 'blue': 2},
 {'shirt': 0, 'jeans': 1, 'dress': 2, 'shoes': 3})

In [11]:
# 将所有图像的标签，转为对应的索引：最终的标签！
all_color_labels = [ pure_color_labels_to_index.get(label) for label in all_color_labels ]
all_item_labels = [ pure_item_labels_to_index.get(label) for label in all_item_labels ]

In [12]:
all_color_labels[0:20], all_item_labels[0:20]

([2, 2, 0, 0, 2, 1, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 0],
 [2, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 2, 2, 2, 1, 0, 3, 2, 0, 2])

后面会用到的变量：
- all_image_path：所有文件的地址
- all_color_labels、all_item_labels：所有文件的颜色、类别标签（索引值）

### 数据预处理：用函数来完成

In [13]:
# 对所有图片预处理：暂时不用图像增强！
def pre_processing_all_image( path ):
    # 预处理：
    image = tf.io.read_file( path )                       # 文件读入
    image = tf.image.decode_jpeg( image, channels = 3 )   # 解码为图片
    image = tf.image.resize( image, [224, 224] )          # 图片划定统一大小
    image = tf.cast( image, tf.float32 )                  # 转换数据类型
    image = image / 255                                   # [0,1]范围内
    image = image * 2 - 1                                 # [-1,1]范围内
    return image

### tf.data获得输入数据：

In [33]:
# 用tf.data创建输入数据集：
AUTOTUNE = tf.data.experimental.AUTOTUNE   # 新操作：在tf.data模块使用时，会自动根据cpu来情况进行并行计算处理！

# 数据提取 + 预处理：
all_images = tf.data.Dataset.from_tensor_slices( all_image_path )  # 先转成张量！
all_images = all_images.map( pre_processing_all_image, num_parallel_calls = AUTOTUNE )
# 因为有双标签，所以标签要先合并到一起：
all_labels = tf.data.Dataset.from_tensor_slices( (all_color_labels, all_item_labels) )  # 先转为张量！—— 其实就是一起弄而已

# 最后的数据集：zip两个张量数据的合并
all_dataset = tf.data.Dataset.zip( (all_images, all_labels) )

In [34]:
# 查看一下标签：每个ele是一个元组，好理解！
for ele in all_labels.take(3):
    print( ele[0].numpy(), ele[1].numpy() )

2 2
2 2
0 0


In [35]:
# 查看一下最后数据集：
all_dataset

<ZipDataset shapes: ((224, 224, 3), ((), ())), types: (tf.float32, (tf.int32, tf.int32))>

### 划分训练集、测试集：

In [36]:
total_numbers = len( all_image_path )      # 一共这么多数据
test_numbers = int( total_numbers * 0.2 )
train_numbers = total_numbers - test_numbers

In [37]:
train_dataset = all_dataset.skip( test_numbers )  # 跳过test_numbers这么多数据，其他全要！
test_dataset = all_dataset.take( test_numbers )   # 取test_numbers这么多数据！

### 乱序、划分batch：

In [38]:
BATCH_SIZE = 32

train_dataset = train_dataset.shuffle( buffer_size = train_numbers ).repeat(-1).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

### 建立模型：使用预训练网络（仅框架）

在函数式编程中，所有的模型层都是可调用的！tf.keras.layers.xxx()()

In [39]:
# 不用头、不要权重参数：纯框架！
mobile_net = tf.keras.applications.MobileNetV2( input_shape = (224,224,3), include_top = False )

In [40]:
# 查看一下：
mobile_net.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [41]:
inputs = tf.keras.Input( shape = (224,224,3) )  # 第一层：输入层！
x = mobile_net(inputs)

In [42]:
# 看一下现在输出的结果：每个层都是可调用的！
x.get_shape()

TensorShape([None, 7, 7, 1280])

In [43]:
# 进入Dense层之前，要先池化一下：把维度降维1维张量！
x = tf.keras.layers.GlobalAveragePooling2D()(x)

In [44]:
x.get_shape()

TensorShape([None, 1280])

In [45]:
# 进入Dense层，并分两支：给每个输出层，都用name属性起一个别名！方便后面模型编译时，用“字典”分别给予不同的设置！

# 颜色部分：3分类！
x1 = tf.keras.layers.Dense(1024, activation = 'relu')(x)
out_color = tf.keras.layers.Dense(3, activation = 'relu', name = 'out_color')(x1)  # 最后的输出层加个名字即可！

# 类别部分：4分类！
x2 = tf.keras.layers.Dense(1024, activation = 'relu')(x)
out_item = tf.keras.layers.Dense(4, activation = 'relu', name = 'out_item')(x2)

In [46]:
# 模型创建：组起来！
model = tf.keras.Model( inputs = inputs, outputs = [out_color, out_item] )

In [47]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
mobilenetv2_1.00_224 (Model)    (None, 7, 7, 1280)   2257984     input_4[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 1280)         0           mobilenetv2_1.00_224[1][0]       
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 1024)         1311744     global_average_pooling2d_1[0][0] 
____________________________________________________________________________________________

### 模型配置：编译

In [49]:
model.compile(
    optimizer = 'adam',
    loss = {'out_color':'sparse_categorical_crossentropy', 'out_item':'sparse_categorical_crossentropy'},
    metrics = ['acc']
)

### 模型训练

In [50]:
train_steps = train_numbers // BATCH_SIZE
test_steps = test_numbers // BATCH_SIZE

In [ ]:
model.fit(
    train_dataset,
    epochs = 15, 
    steps_per_epoch = train_steps,
    validation_data = test_dataset,
    validation_steps = test_steps
)

### 模型评估：一次性评估在测试集上的loss和正确率

In [ ]:
model.evaluate( test_dataset, verbose = 0 )  # 不显示每一个的打印过程（那些=不打印）

### 模型预测：model.predict() —— 方式1

In [ ]:
image = pre_processing_all_image(r'D:/dataset/test/pic1.jpg')

In [ ]:
# 因为输入model的数据是一个batch，即4维的！！ 所以要记得扩充维度！（这里很容易错）

In [ ]:
image = np.expand_dims( image, 0 )  

In [ ]:
# 预测：返回对两个标签的结果：整体在一个list中，每个元素又是一个二维array！
pred = model.predict( image )

In [ ]:
# 都已转为str：
pred_color = pure_color_labels_to_index.get( np.argmax(pred[0][0]) )
pred_item = pure_item_labels_to_index( np.argmax(pred[1][0]) )
pred = pred_color + '_' + pred_item

In [ ]:
# 绘图 + 显示预测的标签：图片要先转回到3维，且回到[0,1]之间（在preprocessing中被转为了[-1,1]）！
image = image[0]
image = (image + 1) / 2

plt.imshow( image )
plt.xlabel( pred )

### 模型预测：model(image, training = False) —— 方式2

说明：model本身就是一个“可调用对象”！可直接给它里面输入东西！—— image是3维的即可！无需升到4维！

注意：
- 当training为False时，model(image, training = False) 等同于 model.predict(image)
- 当training为True时，当model中有BatchNormalization层时会有所不同！若没正则化层，和model.predict()也是一样的！

In [ ]:
model(image, training = False)